# 卷积神经网络CNN - 莫烦pytorch

## 4.1 卷积神经网络处理MINST

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision #包括了一些数据库
import matplotlib.pyplot as plt

In [2]:
# Hyper Parameters
EPOCH = 1
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNIST = False

# MINST数据下载（训练数据）
train_data = torchvision.datasets.MNIST( #去MINST网站下载
    root = './minist',
    train = True, #是否是训练数据
    transform = torchvision.transforms.ToTensor(), #原始数据转换为tensor格式，从图片的(0,255)压缩到(0,1)
    download=DOWNLOAD_MNIST, #如果已经下载好了数据，则设置为False不再重复下载
)

print(train_data.train_data.size()) # (60000, 28, 28)
print(train_data.train_labels.size()) # (60000)
# 画图显示数据（此处只显示第一张图片）
plt.imshow(train_data.train_data[0].numpy(), cmap='gray')
plt.title('%i' % train_data.train_labels[0])
plt.show()

RuntimeError: Dataset not found. You can use download=True to download it

In [ ]:
# 数据装载到Data Loader中得到mini-batch training, the image batch shape will be (50, 1, 28, 28)
train_loader = Data.DataLoader(
    dataset = train_data,
    batch_size = BATCH_SIZE,
    shuffle = True
)

In [ ]:
# MINST数据下载（测试数据）
test_data = torchvision.datasets.MNIST(
    root = './mnist',
    train = False, #不是训练数据，是测试数据
    download = True
)
test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000]/255 #shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_y = test_data.test_labels[:2000]

In [ ]:
# 积(Conv2d) -> 激励函数(ReLU) -> 池化, 向下采样 (MaxPooling) -> 再来一遍 -> 展平多维的卷积成的特征图 -> 接入全连接层 (Linear) -> 输出
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential( #卷积层，使用Sequential定义（和快速定义神经网络一样的操作）
            # 此时图片(1, 28, 28)，分别表示高、长、宽，后两个是图片分辨率
            nn.Conv2d(# 1.卷积层，过滤器（扫描器），长宽指明能够收集多大范围的信息，高度指有多少个filter用来提取出卷积出的特征属性（个数）
                in_channels = 1,   #表示这张图片有多少个（高度）层的，如RGB有三个、灰度图片有一个
                out_channels = 16, #输出的高度，就是filter（卷积核的个数），这16个同时在某个区域上扫描提取了16个不同的特征，然后把他们放在下一层
                kernel_size = 5,   #filter的宽和高都是5个像素点，所以filter的维度5*5*16
                stride = 1,        #步长，每次卷积核移动的像素大小
                padding = 2,       #在边缘添加多少像素点(全0的)，为了不丢失边缘数据特征。 if stride=1, padding=(kernel_size-1)/2=(5-1)/2
            ),  # -> (16, 28, 28)
            nn.ReLU(), # 2.非线性激活层 -> (16, 28, 28)
            nn.MaxPool2d(kernel_size = 2), # 3.池化层，筛选重要的信息。从更厚的数据中提取，也就是缩小长宽， -> (16, 14, 14)
        )
        
        self.conv2 = nn.Sequential( #第二个卷积层
            # 此时图片(16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2), #这一卷积层的in_channel是上一层的out_channel，->(32, 14, 14)
            nn.ReLU(), # -> (32, 14, 14)
            nn.MaxPool2d(2), # ->(32, 7, 7)
        )
        
        self.out = nn.Linear(32 * 7 * 7, 10) #输出层，输入的是图片经过两层卷积后的特征(32,7,7)展平后的结果，输出的是0~9十个分类中的哪个
        #Linear的第三个参数bias表示图层是否学习附加偏差
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x) #(batch, 32, 7, 7)
        x = x.view(x.size(0), -1) #展平数据：(batch, 32*7*7)，保留size(0),-1表示后面维度的数据全部变到一起
        output = self.out(x)
        return output,x

In [ ]:
cnn = CNN()

print(cnn)

In [ ]:
# 训练过程，使用Adam优化
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                       # 交叉熵损失函数，the target label is not one-hotted

# training and testing
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):   # 分配batch data, normalize x when iterate train_loader
        output = cnn(b_x)[0]               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss, calculate the loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients, optimize params

        if step % 50 == 0:
            test_output, last_layer = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
            
# print 10 predictions from test data
test_output, _ = cnn(test_x[:10])
pred_y = torch.max(test_output, 1)[1].data.numpy()
print(pred_y, 'prediction number')
print(test_y[:10].numpy(), 'real number')